In [1]:
import pandas as pd
import numpy as np
import scrape_field
import urllib.request
from bs4 import BeautifulSoup
from collections import OrderedDict
import os
import run_stats

pagename = "http://www.pgatour.com/stats/stat."
pagenameend = "html"
ind_var = run_stats.stats

In [2]:
t_id='047'

In [3]:
dk = pd.read_csv('dfs/draftkings.csv')
dk.set_index('Name', inplace=True)
for stat in ind_var.keys():
#     page = pagename + str(stat) + '.' + pagenameend
    page = pagename + str(stat) + '.y' + str(2021) + '.' + pagenameend #using last year
    try:
        dk[stat] = pd.DataFrame.from_dict(run_stats.createdict(page), orient='index')
    except:
        print (run_stats.stats[stat], str(stat) + ' No good')
        continue
dk.head()

,Position,Name + ID,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,102,103,...,327,328,329,330,352,399,459,460,461,467
Name,,,,,,,,,,,,,,,,,,,,,
Brooks Koepka,G,Brooks Koepka (19543497),19543497,G,11100,Shriners Children's Open,Golf,68.42,56.63,68.01,...,55.49,65.03,71.81,84.24,24.33,9.70,14.54,15.79,4.99,0.466
Sam Burns,G,Sam Burns (19543498),19543498,G,10900,Shriners Children's Open,Golf,70.46,58.90,67.79,...,57.02,63.27,70.39,80.88,25.33,10.13,11.63,16.60,6.46,0.522
Viktor Hovland,G,Viktor Hovland (19543499),19543499,G,10600,Shriners Children's Open,Golf,81.79,63.86,68.61,...,61.23,61.30,70.87,84.77,24.83,9.93,11.88,13.95,3.00,0.537
Abraham Ancer,G,Abraham Ancer (19543500),19543500,G,10400,Shriners Children's Open,Golf,74.31,71.05,69.27,...,56.29,69.14,73.27,85.68,23.64,9.84,7.00,13.46,2.66,0.474
Webb Simpson,G,Webb Simpson (19543501),19543501,G,10200,Shriners Children's Open,Golf,76.55,67.08,69.37,...,65.26,67.29,74.59,87.01,24.85,9.73,14.10,10.58,4.35,0.536


In [4]:
df = pd.read_csv('table_data/tournament_plyr.csv', header=None)
df = df.loc[df[1] == int(t_id)]
df.head()
df.columns = ['p_id','t_id','tourn','year','id','5','6','name','8','place','score','11','lname','13']
df = df.loc[df['year'] >= 2018]
df['MadeCut'] = 0
df.loc[df['score'] > 240, 'MadeCut'] = 1
df[list(ind_var.keys())] = np.NaN
year_dfs = {}
for year in df['year'].unique():
    if year >= 2018:
        year_dfs[year]={}
        for stat in ind_var.keys():
#             print(year, stat)
            page = pagename + str(stat) + '.y' + str(year) + '.' + pagenameend
            try:
                year_dfs[year][stat] = run_stats.createdict(page)
            except:
                print (run_stats.stats[stat], str(stat) + ' No good')
                continue

In [5]:
for year in year_dfs.keys():
    for stat in year_dfs[year].keys():
        for name in year_dfs[year][stat].keys():
            df.loc[(df['year']==year)&(df['name']==name), stat] = year_dfs[year][stat][name]

In [6]:
df.head()

,p_id,t_id,tourn,year,id,5,6,name,8,place,...,327,328,329,330,352,399,459,460,461,467
40179,40180,47,Shriners Hospitals,2021,27936,1,r,Martin Laird,0,1,...,53.57,66.79,78.89,83.79,20.56,10.45,7.37,14.52,3.94,0.581
40180,40181,47,Shriners Hospitals,2021,46435,0,r,Austin Cook,0,2,...,50.70,65.45,68.89,83.08,18.90,10.38,14.32,10.77,4.00,0.484
40181,40182,47,Shriners Hospitals,2021,56278,0,r,Matthew Wolff,0,2,...,49.65,61.88,75.84,81.40,22.42,10.40,16.79,17.50,7.32,0.600
40182,40183,47,Shriners Hospitals,2021,45526,4,r,Abraham Ancer,0,4,...,56.29,69.14,73.27,85.68,23.64,9.84,7.00,13.46,2.66,0.474
40183,40184,47,Shriners Hospitals,2021,32448,0,r,James Hahn,0,5,...,61.27,64.98,74.26,82.98,22.54,10.31,11.02,12.41,4.18,0.521


In [7]:
import statsmodels.api as sm
import statsmodels.formula.api as sm2
import scipy

In [8]:
df_mc = df.loc[df['MadeCut']==1]
df_mc = df_mc.replace({'None': np.NaN})
df_mc = df_mc[['place', *list(ind_var.keys())]]
df_mc.rename(columns=run_stats.stats, inplace=True)
result = sm2.ols(formula="place~"+'+'.join(df_mc.columns[1:]), data=df_mc).fit()
print (result.summary())

                            OLS Regression Results                            
Dep. Variable:                  place   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     1.964
Date:                Wed, 06 Oct 2021   Prob (F-statistic):            0.00167
Time:                        13:16:53   Log-Likelihood:                -1151.2
No. Observations:                 265   AIC:                             2376.
Df Residuals:                     228   BIC:                             2509.
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    2.84e+04   2.06e+04      1.379      0.1

In [9]:
x = df_mc[df_mc.columns[1:]]
y = df_mc['place']
model = sm.OLS(y, x, missing='drop').fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  place   R-squared (uncentered):                   0.782
Model:                            OLS   Adj. R-squared (uncentered):              0.747
Method:                 Least Squares   F-statistic:                              22.75
Date:                Wed, 06 Oct 2021   Prob (F-statistic):                    3.67e-57
Time:                        13:16:53   Log-Likelihood:                         -1152.3
No. Observations:                 265   AIC:                                      2377.
Df Residuals:                     229   BIC:                                      2505.
Df Model:                          36                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
ACCURATE      -0.0514      1.719     -0.030      0.976      -3.438       3.335
GIR            1.3453      3.109      0.433      0.666      -4.780       7.471
SAND          -0.1978      0.297     -0.666      0.506      -0.783       0.388
BIRD_3         2.1093      2.093      1.008      0.315      -2.015       6.233
BIRD_4         7.4773      5.058      1.478      0.141      -2.490      17.444
BIRD_5         1.2556      1.715      0.732      0.465      -2.124       4.635
PUTTS        -25.7478     13.656     -1.885      0.061     -52.655       1.160
SCORE         10.2309      5.933      1.724      0.086      -1.460      21.922
DRIVE         -0.0485      0.084     -0.579      0.563      -0.213       0.116
PAR3         -37.4341     58.866     -0.636      0.525    -153.421      78.553
PAR4          68.0014    120.297      0.565      0.572    -169.029     305.032
PAR5        -100.7069     68.179     -1.477      0.141    -235.046      33.632
PUTT3         -1.2431      1.334     -0.932      0.352      -3.871       1.385
PUTT3_20       0.6984      0.896      0.779      0.437      -1.068       2.465
PUTT3_25       1.8632      1.055      1.766      0.079      -0.216       3.942
B_AVG         17.2589     36.829      0.469      0.640     -55.308      89.826
STRIKE         0.0074      0.057      0.129      0.897      -0.105       0.120
GIR_F          0.6807      0.937      0.726      0.468      -1.166       2.527
LOW            2.3115      2.391      0.967      0.335      -2.399       7.022
FAIRWAY       -2.7072      1.947     -1.391      0.166      -6.543       1.128
D300           1.4487      6.174      0.235      0.815     -10.717      13.615
D280           1.3773      6.214      0.222      0.825     -10.866      13.621
D260           1.9604      6.238      0.314      0.754     -10.331      14.252
D240           2.2144      6.346      0.349      0.727     -10.289      14.718
D220           1.0901      6.318      0.173      0.863     -11.358      13.539
GIR_200       -0.4531      0.363     -1.248      0.213      -1.168       0.262
GIR_175       -0.0009      0.439     -0.002      0.998      -0.866       0.865
GIR_150        0.0800      0.458      0.175      0.862      -0.822       0.982
GIR_125       -0.2804      0.430     -0.653      0.515      -1.127       0.566
GIR_100       -0.9251      0.729     -1.268      0.206      -2.362       0.512
BIRD         -15.7418     10.494     -1.500      0.135     -36.418       4.934
PUTT2         32.8947     13.046      2.521      0.012       7.189      58.600
LEFT          -3.1740      1.405     -2.259      0.025      -5.942      -0.406
RIGHT         -2.9769      1.477     -2.015      0.045      -5.887      -0.066
MISSED        -1.8281      1.822     -1.003      0.317      -

In [10]:
dk.rename(columns=run_stats.stats, inplace=True)
new_x = dk[df_mc.columns[1:]] #no scram at end
dk.loc[:,'place'] = model.predict(new_x)

In [11]:
dk.sort_values('place').head(20)

,Position,Name + ID,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,ACCURATE,GIR,...,GIR_150,GIR_125,GIR_100,BIRD,PUTT2,LEFT,RIGHT,MISSED,SCRAM,place
Name,,,,,,,,,,,,,,,,,,,,,
Webb Simpson,G,Webb Simpson (19543501),19543501,G,10200,Shriners Children's Open,Golf,76.55,67.08,69.37,...,67.29,74.59,87.01,24.85,9.73,14.10,10.58,4.35,0.536,8.071679
Ian Poulter,G,Ian Poulter (19543549),19543549,G,7000,Shriners Children's Open,Golf,59.14,66.33,60.88,...,63.40,61.54,79.68,20.45,9.24,9.31,14.15,4.19,0.370,12.518749
Patrick Reed,G,Patrick Reed (19543513),19543513,G,8600,Shriners Children's Open,Golf,68.50,63.21,62.11,...,60.08,67.04,81.87,23.01,9.42,13.14,12.27,2.74,0.374,12.601244
Brooks Koepka,G,Brooks Koepka (19543497),19543497,G,11100,Shriners Children's Open,Golf,68.42,56.63,68.01,...,65.03,71.81,84.24,24.33,9.70,14.54,15.79,4.99,0.466,12.818879
Aaron Wise,G,Aaron Wise (19543521),19543521,G,7900,Shriners Children's Open,Golf,63.73,60.04,70.35,...,65.18,77.99,86.02,23.47,10.13,12.16,14.43,5.67,0.522,15.422138
Doug Ghim,G,Doug Ghim (19543559),19543559,G,6900,Shriners Children's Open,Golf,61.11,68.55,69.64,...,62.60,77.74,84.25,23.13,9.90,11.76,10.37,3.19,0.493,15.566435
Scottie Scheffler,G,Scottie Scheffler (19543504),19543504,G,9700,Shriners Children's Open,Golf,71.86,63.49,68.08,...,63.18,70.13,83.75,24.89,10.01,16.74,10.13,4.81,0.471,15.732119
Charley Hoffman,G,Charley Hoffman (19543518),19543518,G,8100,Shriners Children's Open,Golf,68.57,57.11,67.85,...,61.09,73.77,80.47,23.91,9.81,12.47,17.42,6.42,0.614,16.311795
Si Woo Kim,G,Si Woo Kim (19543509),19543509,G,9000,Shriners Children's Open,Golf,58.82,62.71,65.78,...,66.54,70.39,81.54,22.22,9.87,10.74,16.54,3.98,0.423,17.369758
